### The following notebook was used to unblind and compile all of the S1 data

In [1]:
# Importing the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib as plb
import seaborn as sns
from scipy import stats
import statistics
import dabest as db

#### Reading in and cleaning up blinded metadata and compound key

In [2]:
md = pd.read_csv('C:/Users/Emily/Documents/S1/metadata/S1_metadata.csv', delimiter=',', encoding='utf-8-sig')
md['Compound library ID'] = md['Compound library ID'].str[-1]
#md.drop(md.columns[[3, 4, 12, 13, 16, 17, 18, 19, 20, 21]], axis = 1, inplace = True)
md = md.drop(md.index[0], axis=0)
md.head()

Date: Recorder Plate ID  \
1  8/23/21    Hodan  S1_R1_1   
2  8/23/21    Hodan  S1_R1_2   
3  8/23/21    Hodan  S1_R1_3   
4  8/23/21    Hodan  S1_R1_4   
5  8/23/21    Hodan  S1_R1_5   

  Different strains in each well of a single assay plate? (Y/N)  \
1                                                NaN              
2                                                NaN              
3                                                NaN              
4                                                NaN              
5                                                NaN              

  Different compounds in each well of a single assay plate? (Y/N)  \
1                                                NaN                
2                                                NaN                
3                                                NaN                
4                                                NaN                
5                                                NaN                

  Compound library ID Compound Well A Compound Well B Compound Well C  \
1                   1              B2              B3              B4   
2                   1              C2              C3              C4   
3                   1              D2              D3              D4   
4                   1              E2              E3              E4   
5                   1              F2              F3              F4   

  Compound Well D  ...  Chemotaxis Start (24 hrs format)  \
1              B5  ...                             11:10   
2              C5  ...                             11:10   
3              D5  ...                             11:10   
4              E5  ...                             11:10   
5              F5  ...                             11:10   

   Chemotaxis End (24 hrs format) Image ID Scanner Slot: Scanner Slot  \
1                           12:10   S1_001             1          NaN   
2                           12:10   S1_001             2          NaN   
3                           12:10   S1_001             3          NaN   
4                           12:10   S1_001             4          NaN   
5                           12:10   S1_002             1          NaN   

   Scanner #  Strain Well A   Strain Well B Strain Well C Strain Well D  
1          3              N2             N2            N2            N2  
2          3              N2             N2            N2            N2  
3          3              N2             N2            N2            N2  
4          3              N2             N2            N2            N2  
5          4              N2             N2            N2            N2  

[5 rows x 22 columns]

In [3]:
compound_key = pd.read_csv('C:/Users/Emily/Documents/S1/metadata/S1_randomized_compounds.csv', index_col=0)
compound_key['Plate Number'] =  compound_key['Plate Number'].str.strip().str[-1]
compound_key.head()


CAS ID                               Compound  Num Let Compound Well  \
0     9/2/5451  5-Aminolevulinic acid (hydrochloride)    2   B            B2   
1          NaN                                Alyssin    3   B            B3   
2     112-39-0                       Methyl palmitate    4   B            B4   
3     116-26-7                               Safranal    5   B            B5   
4  168316-95-8                               Spinosad    2   C            C2   

  Plate Number  
0            1  
1            1  
2            1  
3            1  
4            1

#### Generating functions to unblind the compounds in the metadata

In [4]:
def add_compoundA(row, compound_map ):
    compound = compound_map.loc[
        (compound_map['Plate Number']==row['Compound library ID']) & 
        (compound_map['Compound Well']==row['Compound Well A'])]['Compound']
    return compound.values[0]

def add_compoundB(row, compound_map ):
    compound = compound_map.loc[
        (compound_map['Plate Number']==row['Compound library ID']) & 
        (compound_map['Compound Well']==row['Compound Well B'])]['Compound']
    #print(compound.values)
    return compound.values[0]

def add_compoundC(row, compound_map ):
    compound = compound_map.loc[
        (compound_map['Plate Number']==row['Compound library ID']) & 
        (compound_map['Compound Well']==row['Compound Well C'])]['Compound']
    return compound.values[0]

def add_compoundD(row, compound_map ):
    compound = compound_map.loc[
        (compound_map['Plate Number']==row['Compound library ID']) & 
        (compound_map['Compound Well']==row['Compound Well D'])]['Compound']
    return compound.values[0]

In [5]:
md['Compound A'] = md.apply(
    lambda row: add_compoundA(row, compound_key), axis=1)

md['Compound B'] = md.apply(
    lambda row: add_compoundB(row, compound_key), axis=1)
    
md['Compound C'] = md.apply(
    lambda row: add_compoundC(row, compound_key), axis=1)
    
md['Compound D'] = md.apply(
    lambda row: add_compoundD(row, compound_key), axis=1)

md['Scanner Slot:'] = md['Scanner Slot:'].apply(str)

#### Reading in the Image Analysis summary file. 
<p> The metadata sheet will be used to fill in missing fields on the Image Analysis summary file including Compound, Strain and Plate ID </p>

In [6]:
ia_data = pd.read_csv('C:/Users/Emily/Documents/S1/image_analysis/All_reps.csv', index_col=0)
ia_data = ia_data.drop(['Large Object'], axis=1)

#### Generating a function to add the plate ID and compound name to the summary data

In [7]:
def add_PlateID(row, metadata):
    slotID = row['WellNo'][0]
    pid = metadata.loc[
        (metadata['Image ID']==row['File Name']) & 
        (metadata['Scanner Slot:']==slotID)]['Plate ID']
    return pid.values[0]

In [8]:
ia_data['Plate ID'] = ia_data.apply(
    lambda row: add_PlateID(row, md), axis=1)

In [9]:
def add_Compound(row, metadata):
    wellID = row['WellNo'][1]
    if wellID == 'A':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound A']
    elif wellID == 'B':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound B']
    elif wellID == 'C':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound C']
    elif wellID == 'D':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound D']
    print(compound)
    return compound.values[0]

In [10]:
ia_data['Compound'] = ia_data.apply(
    lambda row: add_Compound(row, md), axis=1)

1    5-Aminolevulinic acid (hydrochloride)
Name: Compound A, dtype: object
1    Alyssin
Name: Compound B, dtype: object
1    Methyl palmitate
Name: Compound C, dtype: object
1    Safranal
Name: Compound D, dtype: object
2    Spinosad
Name: Compound A, dtype: object
2    Ellagic acid
Name: Compound B, dtype: object
2    Camphor
Name: Compound C, dtype: object
2    Citronellol
Name: Compound D, dtype: object
3    Rosmarinic acid
Name: Compound A, dtype: object
3    Kaempferol
Name: Compound B, dtype: object
3    Piperonyl Alcohol
Name: Compound C, dtype: object
3    Coumaran
Name: Compound D, dtype: object
4    Diacetyl
Name: Compound A, dtype: object
4    Guaiazulene
Name: Compound B, dtype: object
4    (-)-Huperzine A
Name: Compound C, dtype: object
4    L-Mimosine
Name: Compound D, dtype: object
5    Micheliolide
Name: Compound A, dtype: object
5    Beta caryophyllene
Name: Compound B, dtype: object
5    Salicylic acid
Name: Compound C, dtype: object
5    Nerolidol
Name: Compound D, d

64    Cinnamyl Alcohol
Name: Compound C, dtype: object
64    2,3-Dihydrobenzofuran
Name: Compound D, dtype: object
53    Ajmalicine
Name: Compound A, dtype: object
53    Galanthamine
Name: Compound B, dtype: object
53    Bergapten
Name: Compound C, dtype: object
53    Eucalyptol
Name: Compound D, dtype: object
54    Ursolic acid
Name: Compound A, dtype: object
54    Solasodine
Name: Compound B, dtype: object
54    Anisole
Name: Compound C, dtype: object
54    trans-Cinnamaldehyde
Name: Compound D, dtype: object
55    Carvacrol
Name: Compound A, dtype: object
55    Phytol
Name: Compound B, dtype: object
55    p-Anisic acid
Name: Compound C, dtype: object
55    2-nonanone
Name: Compound D, dtype: object
56    Curcumenol
Name: Compound A, dtype: object
56    H2O
Name: Compound B, dtype: object
56    α-Phellandrene
Name: Compound C, dtype: object
56    DMSO
Name: Compound D, dtype: object
57    Piperitenone
Name: Compound A, dtype: object
57    Nootkatone
Name: Compound B, dtype: object
57

#### Filtering the dataset to only include replicates with 150 worms or more
<p> We also want to identify any compounds with fewer than 3 biological replicates with over 150 worms<p>

In [11]:
ia_data = ia_data.loc[ia_data['Total Worms']>= 150] 
ia_data['comp_count'] = ia_data.groupby('Compound')['Compound'].transform('count')
ia_data.loc[ia_data['comp_count']<3] 

WellNo  Total Worms  Chemotaxis            Compound  Strain File Name  \
0      1A        329.0    0.212121        Paeoniflorin     NaN    S1_005   
2      1C        313.0    0.828179     Isoamyl alcohol     NaN    S1_005   
10     3C        268.0    0.451327     Phenylacetylene     NaN    S1_011   
8      3A        227.0    0.431472        Paeoniflorin     NaN    S1_009   
10     3C        189.0    0.847059     Isoamyl alcohol     NaN    S1_009   
2      1C        228.0    0.215311     Phenylacetylene     NaN    S1_021   
3      1D        191.0    0.584270  2-Methyl-1-butanol     NaN    S1_017   
11     3D        190.0    0.780347  2-Methyl-1-butanol     NaN    S1_012   

    Well width  Plate ID  comp_count  
0       3039.0  S1_R1_17           2  
2       3030.0  S1_R1_17           2  
10      3058.0  S1_R2_19           2  
8       3050.0  S1_R2_11           2  
10      3066.0  S1_R2_11           2  
2       3024.0   S1_R4_7           2  
3       3017.0  S1_R3_17           2  
11      3055.0  S1_R2_23           2

#### Reading in all of the files that contain worm positions based on the exclusion criteria above

In [12]:
wrm_locs_fldr = plb.Path('C:/Users/Emily/Documents/S1/image_analysis/')

In [13]:
def get_worm_locs(row, wrms, result_dict): 

    fname = row['File Name']
    wellnum = row['WellNo']
    loc_fname =  wrms.joinpath('loc_' + fname + '_' + wellnum + '.csv')
    temp = pd.read_csv(loc_fname)
    compound = row['Compound']
    xs = temp['X']
    #xs = list(temp['centroid-1'])
    if compound in result_dict:
        result_dict[compound] = result_dict[compound].append(xs)
        result_dict[compound].reset_index(inplace=True, drop=True)
        
        #result_dict[compound] = result_dict[compound]+xs
    else:
        result_dict[compound]=xs
    
    return result_dict

In [14]:
ia_data = ia_data.sort_values(['Compound', 'File Name'], ascending=[True, True])


#### To perform statistical analyses we need to pool all of the worm positiions from each biological replicate for each condition.

<p> We also want to capture the first 3 biological replicates performed for each condition. Some conditions were captured 4 times due to errors made during the screening process <p>

In [22]:
# Need to create an empty dictionary to hold the values
results_dict = {}
compound = ''
i=0

for index, row in ia_data.iterrows():
    if row['Compound'] == compound:
        i += 1
        compound = row['Compound']
        if i < 3:
            pooled = get_worm_locs(row, wrm_locs_fldr, results_dict)
        else:
            continue
    else:
        i = 0
        compound = row['Compound']
        pooled = get_worm_locs(row, wrm_locs_fldr, results_dict)

    
#Remember that Dabest requires a dataframe. Casting dict to df
pooled_df = pd.DataFrame.from_dict(pooled)

#### Converting the worm locations from measurements in dots per inch to millimeters

In [23]:
# 1 inch = 25.4mm
mm = 25.4
# 1200 pixels per 25.4mm
px_mm = 1200/mm

#The following transforms the worm location data so that it is respective to the start zone
# The start zone is at the center of the image; Start Zone = 0mm
#Worms with positive values are closer to the compound, negative values are away from the compound
mm_df = pooled_df.apply(lambda x: -(x/px_mm)+32.5)
#mm_df.to_csv('C:/Users/Emily/Documents/S1/S1_xs3.csv')

#### Creating an ordered list of compounds to pass to Dabest to calculate confidence intervals
<p> The control condition always needs to be the first item in the list

In [24]:
sums = ia_data.groupby('Compound').mean().reset_index()
ia_sort = sums.copy()
condition = (ia_sort.Compound=='DMSO') | (ia_sort.Compound=='H2O')

excluded = ia_sort[condition]
included = ia_sort[~condition]
sortd = included.sort_values(by='Chemotaxis',ascending=True)
ia_sort = pd.concat([excluded, sortd])

cmpd_ordr = ia_sort['Compound'].to_list()

#### Generating confidence intervals using DMSO as the control group

In [25]:
db_obj = db.load(mm_df, idx=(cmpd_ordr))
pooled_mm_obj = db.load(mm_df, idx=(cmpd_ordr))
results_df_mm = pooled_mm_obj.mean_diff.results

#### Filtering the DMSO analysis for compounds with confidence intervals that do not span the mean of position of worms against DMSO

In [28]:
same = results_df_mm.loc[(results_df_mm['bca_low'] < 0) & (results_df_mm['bca_high'] > 0)]
same_comps = same['test'].to_list()
diff = results_df_mm[~results_df_mm['test'].isin(same_comps)]
diff = diff.append(results_df_mm.loc[results_df_mm['test']=='H2O'])
diff = diff.sort_values(['difference'])
diff

control                       test  control_N  test_N      effect_size  \
1     DMSO                  1-octanol       1068     652  mean difference   
2     DMSO                     Phytol       1068     625  mean difference   
4     DMSO                 2-nonanone       1068     867  mean difference   
3     DMSO               Ellagic acid       1068     890  mean difference   
6     DMSO    Salvinorin A Propionate       1068     926  mean difference   
7     DMSO                   Spinosad       1068     900  mean difference   
8     DMSO                    Camphor       1068     941  mean difference   
10    DMSO               Ursolic acid       1068     884  mean difference   
19    DMSO                   Safranal       1068     848  mean difference   
5     DMSO           Methyl palmitate       1068     935  mean difference   
25    DMSO   Ethyl p-methoxycinnamate       1068     751  mean difference   
12    DMSO                Daucosterol       1068     804  mean difference   
9     DMSO             Oleanolic Acid       1068     730  mean difference   
0     DMSO                        H2O       1068     915  mean difference   
71    DMSO  2-Nonylquinolin-4(1H)-one       1068     803  mean difference   
67    DMSO                  Nerolidol       1068     781  mean difference   
61    DMSO      4-Methoxybenzaldehyde       1068     901  mean difference   
74    DMSO                Salsolidine       1068     748  mean difference   
55    DMSO                   Apigenin       1068     754  mean difference   
51    DMSO            Geranyl Acetate       1068     890  mean difference   
65    DMSO              β-Citronellol       1068     860  mean difference   
72    DMSO                   Piperine       1068     787  mean difference   
56    DMSO                Pellitorine       1068     846  mean difference   
68    DMSO                (-)-Cedrene       1068    1036  mean difference   
58    DMSO   Sinomenine hydrochloride       1068     883  mean difference   
46    DMSO                 Cadaverine       1068     788  mean difference   
54    DMSO                   Sabinene       1068     823  mean difference   
66    DMSO          Piperonyl Alcohol       1068    1008  mean difference   
53    DMSO                   Lapachol       1068     811  mean difference   
78    DMSO             p-Tolualdehyde       1068     873  mean difference   
75    DMSO  2,5-Dihydroxybenzoic acid       1068     971  mean difference   
64    DMSO                   Carnosol       1068     930  mean difference   
59    DMSO            Ethyl palmitate       1068     887  mean difference   
79    DMSO               Piperitenone       1068     951  mean difference   
69    DMSO           Cinnamyl Alcohol       1068     857  mean difference   
70    DMSO                    Limonin       1068     907  mean difference   
73    DMSO                    Anisole       1068     858  mean difference   
80    DMSO            (-)-Huperzine A       1068     911  mean difference   
76    DMSO                   Furfural       1068     796  mean difference   
81    DMSO               Isoquinoline       1068     836  mean difference   
77    DMSO                 Solasodine       1068     826  mean difference   
84    DMSO                 L-Mimosine       1068     916  mean difference   
82    DMSO                Guaiazulene       1068     958  mean difference   
83    DMSO                  Leonurine       1068     766  mean difference   
89    DMSO                   Coumaran       1068     754  mean difference   
85    DMSO               Acetophenone       1068     755  mean difference   
88    DMSO             α-Phellandrene       1068     777  mean difference   
86    DMSO               Paeoniflorin       1068     556  mean difference   
87    DMSO            Phenylacetylene       1068     496  mean difference   
91    DMSO                   Diacetyl       1068     796  mean difference   
90    DMSO      2,3-Dihydrobenzofuran       1068     705  mean difference   

#### Swapping the list order so that water is the control group

In [29]:
def swap_rows(df, i1, i2):
    a, b = df.iloc[0], df.iloc[1]
    df.iloc[i1, :], df.iloc[i2, :] = b, a
    return df

swapped = swap_rows(excluded, 0, 1)
h20_analysis = swapped.append(included)
h20_analysis.tail()

Compound  Total Worms  Chemotaxis  Strain   Well width  \
91         p-Anisic acid   341.333333   -0.076448     NaN  3046.333333   
92        p-Tolualdehyde   291.000000    0.189825     NaN  3039.666667   
93  trans-Cinnamaldehyde   293.000000    0.019462     NaN  3051.666667   
94        α-Phellandrene   259.000000    0.341553     NaN  3045.333333   
95         β-Citronellol   260.750000    0.122218     NaN  3038.000000   

    comp_count  
91           3  
92           3  
93           3  
94           3  
95           4

#### Calculating confidence intervals using water as the control group

In [30]:
h20_ordr = h20_analysis['Compound'].to_list()
h20_obj = db.load(mm_df, idx=(h20_ordr))
h20_md_results = h20_obj.mean_diff.results
h20_md_results

control                  test  control_N  test_N      effect_size  \
0      H2O                  DMSO        915    1068  mean difference   
1      H2O           (+)-Carvone        915    1052  mean difference   
2      H2O           (-)-Borneol        915     867  mean difference   
3      H2O           (-)-Cedrene        915    1036  mean difference   
4      H2O       (-)-Huperzine A        915     911  mean difference   
..     ...                   ...        ...     ...              ...   
90     H2O         p-Anisic acid        915    1024  mean difference   
91     H2O        p-Tolualdehyde        915     873  mean difference   
92     H2O  trans-Cinnamaldehyde        915     879  mean difference   
93     H2O        α-Phellandrene        915     777  mean difference   
94     H2O         β-Citronellol        915     860  mean difference   

    is_paired  difference  ci   bca_low  bca_high  ... resamples  random_seed  \
0       False   -0.418416  95 -1.835136  1.079114  ...      5000        12345   
1       False    0.703972  95 -0.733226  2.151434  ...      5000        12345   
2       False   -1.210954  95 -2.782015  0.362023  ...      5000        12345   
3       False    1.584452  95 -0.021242  3.080056  ...      5000        12345   
4       False    2.959551  95  1.392195  4.435206  ...      5000        12345   
..        ...         ...  ..       ...       ...  ...       ...          ...   
90      False   -1.586643  95 -3.142784 -0.052098  ...      5000        12345   
91      False    1.987114  95  0.397433  3.483390  ...      5000        12345   
92      False   -0.375126  95 -1.989382  1.223275  ...      5000        12345   
93      False    4.949155  95  3.271456  6.519793  ...      5000        12345   
94      False    1.486028  95 -0.115474  2.990822  ...      5000        12345   

    pvalue_permutation permutation_count  pvalue_welch  statistic_welch  \
0               0.5740              5000  5.824984e-01         0.549831   
1               0.3566              5000  3.498650e-01        -0.935080   
2               0.1438              5000  1.380134e-01         1.483902   
3               0.0408              5000  4.176785e-02        -2.037181   
4               0.0002              5000  1.086005e-04        -3.879056   
..                 ...               ...           ...              ...   
90              0.0438              5000  4.107570e-02         2.044140   
91              0.0108              5000  1.113149e-02        -2.541192   
92              0.6526              5000  6.412641e-01         0.466011   
93              0.0000              5000  1.560789e-09        -6.072308   
94              0.0494              5000  5.344987e-02        -1.932558   

    pvalue_students_t  statistic_students_t  pvalue_mann_whitney  \
0        5.826499e-01              0.549608         5.920424e-01   
1        3.489966e-01             -0.936761         3.573942e-01   
2        1.375771e-01              1.485543         1.374698e-01   
3        4.222333e-02             -2.032649         3.555616e-02   
4        1.088434e-04             -3.878473         2.309168e-04   
..                ...                   ...                  ...   
90       4.136976e-02              2.041164         3.869320e-02   
91       1.120495e-02             -2.538883         1.454251e-02   
92       6.411169e-01              0.466217         6.493164e-01   
93       1.631275e-09             -6.064321         1.582380e-09   
94       5.404872e-02             -1.927726         7.671689e-02   

    statistic_mann_whitney  
0                 495422.0  
1                 469726.0  
2                 412778.5  
3                 447868.0  
4                 375296.5  
..                     ...  
90                493925.5  
91                372731.0  
92                407130.5  
93                295028.0  
94                374347.5  

[95 rows x 25 columns]

In [31]:
#h20_md_results.to_csv('D:/_2021_08_screen/analysis/S1mdiff_150_h2o.csv')

In [41]:
results_df_mm = results_df_mm.append(h20_md_results)
results_df_mm.to_csv('C:/Users/Emily/Documents/S1/S1_3.csv')

#### Filtering the water analysis for compounds with confidence intervals that do not span the mean of position of worms against water

In [33]:
h2o_same = h20_md_results.loc[(h20_md_results['bca_low'] < 0) & (h20_md_results['bca_high'] > 0)]
h2o_same_comps = h2o_same['test'].to_list()
h2o_diff = h20_md_results[~h20_md_results['test'].isin(h2o_same_comps)]
h2o_diff = h2o_diff.append(h20_md_results.loc[h20_md_results['test']=='DMSO'])
h2o_diff = h2o_diff.sort_values(['difference'])
h2o_diff

control                       test  control_N  test_N      effect_size  \
5      H2O                  1-octanol        915     652  mean difference   
67     H2O                     Phytol        915     625  mean difference   
10     H2O                 2-nonanone        915     867  mean difference   
36     H2O               Ellagic acid        915     890  mean difference   
78     H2O    Salvinorin A Propionate        915     926  mean difference   
84     H2O                   Spinosad        915     900  mean difference   
25     H2O                    Camphor        915     941  mean difference   
88     H2O               Ursolic acid        915     884  mean difference   
75     H2O                   Safranal        915     848  mean difference   
57     H2O           Methyl palmitate        915     935  mean difference   
37     H2O   Ethyl p-methoxycinnamate        915     751  mean difference   
33     H2O                Daucosterol        915     804  mean difference   
63     H2O             Oleanolic Acid        915     730  mean difference   
90     H2O              p-Anisic acid        915    1024  mean difference   
0      H2O                       DMSO        915    1068  mean difference   
79     H2O   Sinomenine hydrochloride        915     883  mean difference   
74     H2O                   Sabinene        915     823  mean difference   
70     H2O          Piperonyl Alcohol        915    1008  mean difference   
51     H2O                   Lapachol        915     811  mean difference   
91     H2O             p-Tolualdehyde        915     873  mean difference   
7      H2O  2,5-Dihydroxybenzoic acid        915     971  mean difference   
27     H2O                   Carnosol        915     930  mean difference   
38     H2O            Ethyl palmitate        915     887  mean difference   
69     H2O               Piperitenone        915     951  mean difference   
29     H2O           Cinnamyl Alcohol        915     857  mean difference   
53     H2O                    Limonin        915     907  mean difference   
18     H2O                    Anisole        915     858  mean difference   
4      H2O            (-)-Huperzine A        915     911  mean difference   
40     H2O                   Furfural        915     796  mean difference   
46     H2O               Isoquinoline        915     836  mean difference   
82     H2O                 Solasodine        915     826  mean difference   
49     H2O                 L-Mimosine        915     916  mean difference   
44     H2O                Guaiazulene        915     958  mean difference   
52     H2O                  Leonurine        915     766  mean difference   
31     H2O                   Coumaran        915     754  mean difference   
13     H2O               Acetophenone        915     755  mean difference   
93     H2O             α-Phellandrene        915     777  mean difference   
64     H2O               Paeoniflorin        915     556  mean difference   
66     H2O            Phenylacetylene        915     496  mean difference   
35     H2O                   Diacetyl        915     796  mean difference   
6      H2O      2,3-Dihydrobenzofuran        915     705  mean difference   
86     H2O                  Thiophene        915     706  mean difference   
8      H2O         2-Methyl-1-butanol        915     381  mean difference   
45     H2O            Isoamyl alcohol        915     502  mean difference   

    is_paired  difference  ci    bca_low   bca_high  ... resamples  \
5       False   -7.864674  95  -9.412386  -6.307701  ...      5000   
67      False   -6.667332  95  -8.326055  -4.998785  ...      5000   
10      False   -4.994392  95  -6.570856  -3.472582  ...      5000   
36      False   -4.162435  95  -5.656421  -2.786431  ...      5000   
78      False   -3.731821  95  -5.300717  -2.233021  ...      5000   
84      False   -3.215357  95  -4.791870  -1.771035  ...      5000   
25      False   -3.174772  95  -4.717200  -1.69

#### Merging the water and DMSO analysis to reflect only compounds that are found in both analyses
<p> Compounds found on the resulting dataset will be retested using the the tax-4 null, osm-9 null and tax-4::osm-9 double null mutant strains

In [38]:
inner = pd.merge(h2o_diff, diff , on=["test"])
inner = inner.append(h2o_diff.loc[h2o_diff['test']=='DMSO'])
inner = inner.append(diff.loc[diff['test']=='H2O'])

In [39]:
comps = inner['test']
all_control_results = results_df_mm.append(h20_md_results)
all_control_results = all_control_results.loc[all_control_results['test'].isin(comps)]
print(len(all_control_results))

86


In [40]:
all_control_results.to_csv('C:/Users/Emily/Documents/S1/inner3.csv')